In [9]:
# !pip install -q -U google-genai pydantic

# Building Persona Script Generator with Gemini API
# Install required packages first

# !pip install -q -U google-genai

from google import genai
import os
import json
from datetime import datetime
from typing import Dict, List, Optional
import time

In [2]:
# Initialize the model

client = genai.Client()

# response = client.models.generate_content(
#     model="gemini-3-flash-preview",
#     contents="How does AI work?"
# )
# print(response.text)

## Use database to generate prompt

In [3]:
# Database file path
DB_FILE = "buildings_database.json"

def load_database() -> Dict:
    """Load the buildings database from file."""
    if os.path.exists(DB_FILE):
        with open(DB_FILE, 'r') as f:
            return json.load(f)
    return {}

def save_database(database: Dict) -> None:
    """Save the buildings database to file."""
    with open(DB_FILE, 'w') as f:
        json.dump(database, f, indent=2)

def get_building_data(building_id: str) -> Optional[Dict]:
    """Retrieve building data from database."""
    database = load_database()
    return database.get(building_id)

def update_building_data(building_id: str, building_name: str, data: Dict) -> None:
    """Update or create building data in database with name, persona, and script fields."""
    database = load_database()
    # Ensure the data has name, persona, and script fields
    building_entry = database.get(building_id, {})
    building_entry["name"] = building_name
    building_entry.update(data)
    database[building_id] = building_entry
    save_database(database)

## Generate building personas

In [13]:
from ast import Return


def get_persona(building_id: str, building_name: str) -> str:
    """
    Get or generate persona for a building.
    1. Check if building exists in database
    2. If it exists and has a persona, return it
    3. Otherwise, generate persona using Gemini API, store it, and return
    """
    building_data = get_building_data(building_id)
    
    if building_data and "persona" in building_data and building_data["persona"]:
        return building_data["persona"]
    
    prompt = f"For the building named '{building_name}', based on its building type and usage, historical facts and modern context, generate a persona using exactly 5 descriptive adjectives describing tone, speaking style, and personality traits."
    response = client.models.generate_content(
        model="gemini-3-flash-preview",
        contents=prompt
    )
    persona = response.text
    
    # Store persona in database while preserving existing script
    update_data = {"persona": persona}
    if building_data and "script" in building_data:
        update_data["script"] = building_data["script"]
    else:
        update_data["script"] = ""
    update_building_data(building_id, building_name, update_data)
    
    return persona

## Prompt Gemini to get script

In [14]:
def get_script(building_id: str, building_name: str) -> str:
    """
    Get or generate script for a building.
    1. Check if building exists in database
    2. If it exists and has persona and script, return script
    3. If it exists with persona but no script, generate and store it
    4. If it doesn't exist or has no persona, get persona first then generate script
    """
    building_data = get_building_data(building_id)
    
    if building_data and "script" in building_data and building_data["script"]:
        return building_data["script"]

    # Get existing persona or generate new one
    if building_data and "persona" in building_data and building_data["persona"]:
        persona = building_data["persona"]
    else:
        persona = get_persona(building_id, building_name)
        # Reload building_data after get_persona updates it
        building_data = get_building_data(building_id)

    prompt = f"""Using the persona: {persona}
    
Generate a building script for '{building_name}' that:
- Reflects the persona defined above as if in conversation with a visitor
- Is approximately 2-3 sentences
- Begins with an attention-grabbing welcome statement to the building and area
- Highlights key historical facts and significance
- Emphasizes unique architectural features
- Conveys the building's role and importance in its location
- Includes the building's unique characteristics and stories"""
    
    response = client.models.generate_content(
        model="gemini-3-flash-preview",
        contents=prompt
    )
    script = response.text
    
    # Store script in database while preserving persona
    update_data = {"script": script, "persona": persona}
    update_building_data(building_id, building_name, update_data)
    
    return script

In [15]:
def remove_persona(building_id: str, building_name: str) -> None:
    """Remove persona from building data."""
    building_data = get_building_data(building_id)
    if building_data:
        building_data["persona"] = ""
        update_building_data(building_id, building_name, {"persona": ""})

def remove_script(building_id: str, building_name: str) -> None:
    """Remove script from building data."""
    building_data = get_building_data(building_id)
    if building_data:
        building_data["script"] = ""
        update_building_data(building_id, building_name, {"script": ""})

In [ ]:
file_path = '../data/landmarks.json' 

with open(file_path, 'r') as f:
    landmarks_data = json.load(f)
    
landmarks = landmarks_data['landmarks']

# script = get_script("empire_state_building", "Empire State Building")

    # sleep(1)  # To avoid rate limiting

ClientError: 429 RESOURCE_EXHAUSTED. {'error': {'code': 429, 'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. To monitor your current usage, head to: https://ai.dev/rate-limit. \n* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 20, model: gemini-3-flash\nPlease retry in 46.562678244s.', 'status': 'RESOURCE_EXHAUSTED', 'details': [{'@type': 'type.googleapis.com/google.rpc.Help', 'links': [{'description': 'Learn more about Gemini API quotas', 'url': 'https://ai.google.dev/gemini-api/docs/rate-limits'}]}, {'@type': 'type.googleapis.com/google.rpc.QuotaFailure', 'violations': [{'quotaMetric': 'generativelanguage.googleapis.com/generate_content_free_tier_requests', 'quotaId': 'GenerateRequestsPerDayPerProjectPerModel-FreeTier', 'quotaDimensions': {'location': 'global', 'model': 'gemini-3-flash'}, 'qu